# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298296918538                   -0.85    5.1         
  2   -8.300189288025       -2.72       -1.25    1.0    128ms
  3   -8.300430533449       -3.62       -1.89    3.1    178ms
  4   -8.300462013419       -4.50       -2.77    1.9    149ms
  5   -8.300464166342       -5.67       -3.05    3.1    249ms
  6   -8.300464402295       -6.63       -3.22    3.5    163ms
  7   -8.300464521262       -6.92       -3.38    6.1    245ms
  8   -8.300464588810       -7.17       -3.54    1.4    154ms
  9   -8.300464628182       -7.40       -3.73    2.1    166ms
 10   -8.300464633427       -8.28       -3.81    1.1    138ms
 11   -8.300464642034       -8.07       -4.05    1.5    204ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64160133244                   -0.70    5.8         
  2   -16.67805348929       -1.44       -1.14    1.4    374ms
  3   -16.67919214971       -2.94       -1.88    2.6    468ms
  4   -16.67927760394       -4.07       -2.74    3.1    831ms
  5   -16.67928580020       -5.09       -3.09    6.0    579ms
  6   -16.67928618700       -6.41       -3.49    2.0    363ms
  7   -16.67928621538       -7.55       -3.96    2.1    381ms
  8   -16.67928622146       -8.22       -4.55    3.1    438ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32527558738                   -0.56    6.6         
  2   -33.33266698255       -2.13       -1.00    1.1    1.23s
  3   -33.33403735266       -2.86       -1.74    4.9    1.67s
  4   -33.33427411446       -3.63       -2.62    2.1    1.31s
  5   -33.33574371712       -2.83       -2.51    7.5    2.13s
  6   -33.33694311547       -2.92       -2.52    5.4    1.56s
  7   -33.33694322323       -6.97       -2.52    2.5    1.29s
  8   -33.33674864736   +   -3.71       -2.23    2.2    1.46s
  9   -33.33676508783       -4.78       -2.24    1.0    1.18s
 10   -33.33682403225       -4.23       -2.33    1.0    1.20s
 11   -33.33686568047       -4.38       -2.42    1.0    1.20s
 12   -33.33688287836       -4.76      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298209755356                   -0.85    5.0         
  2   -8.300256459346       -2.69       -1.58    1.2    122ms
  3   -8.300439581161       -3.74       -2.70    3.8    168ms
  4   -8.300451332625       -4.93       -2.69    2.9    184ms
  5   -8.300463585350       -4.91       -3.20    1.0    118ms
  6   -8.300464492486       -6.04       -3.67    3.2    177ms
  7   -8.300464641333       -6.83       -4.52    1.8    176ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32475059812                   -0.56    6.6         
  2   -33.27814567540   +   -1.33       -1.25    1.0    1.05s
  3   +12.14239285684   +    1.66       -0.24    8.6    2.74s
  4   -33.32647874231        1.66       -1.82    6.4    2.52s
  5   -33.23817267351   +   -1.05       -1.35    4.1    1.99s
  6   -33.25862198414       -1.69       -1.60    3.4    1.64s
  7   -32.98156137158   +   -0.56       -1.29    4.8    1.75s
  8   -33.33475543620       -0.45       -2.17    4.1    1.62s
  9   -33.33631209421       -2.81       -2.45    2.1    1.26s
 10   -33.33680045448       -3.31       -2.63    2.9    1.44s
 11   -33.33692440080       -3.91       -3.01    1.6    1.27s
 12   -33.33693626148       -4.93      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.